## 1. Mount Google Drive & Import Libraries

Pada bagian ini kita akan:
- Mount Google Drive untuk mengakses dataset
- Import semua library yang diperlukan untuk EDA, preprocessing, modeling, dan evaluasi

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import Libraries untuk Data Manipulation
import pandas as pd
import numpy as np

# Import Libraries untuk Visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# Import Libraries untuk Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Import Libraries untuk Modeling
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

# Import Libraries untuk Evaluasi
from sklearn.metrics import (
    confusion_matrix, 
    classification_report, 
    roc_auc_score, 
    roc_curve,
    accuracy_score
)

# Import untuk Feature Importance
from xgboost import plot_importance

# Setting untuk visualisasi
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

print("✅ Semua library berhasil diimport!")

---

## 2. Load Data & Exploratory Data Analysis (EDA)

### 2.1 Load Dataset

**Instruksi:** 
- Ganti path di bawah ini sesuai lokasi file `heart_disease.csv` di Google Drive Anda
- Contoh: `/content/drive/MyDrive/ML_Praktikum/Pertemuan14/heart_disease.csv`

In [ ]:
# Load Dataset dari Google Drive
# ⚠️ GANTI PATH INI SESUAI LOKASI FILE ANDA!
dataset_path = '/content/drive/MyDrive/path/to/your/heart_disease.csv'

df = pd.read_csv(dataset_path)

print(f"Dataset berhasil dimuat!")
print(f"Jumlah Baris: {df.shape[0]}")
print(f"Jumlah Kolom: {df.shape[1]}")
print("\n" + "="*50)
df.head()

### 2.2 Informasi Dataset

Melihat struktur data, tipe data setiap kolom, dan keberadaan missing values.

In [ ]:
# Informasi umum dataset
print("=== INFORMASI DATASET ===")
df.info()

print("\n" + "="*50)
print("\n=== STATISTIK DESKRIPTIF ===")
df.describe()

### 2.3 Pengecekan Missing Values

Mengecek apakah ada nilai yang hilang (missing values) di dataset.

In [ ]:
# Cek missing values
missing_values = df.isnull().sum()

print("=== MISSING VALUES ===")
print(missing_values)

if missing_values.sum() == 0:
    print("\n✅ Tidak ada missing values dalam dataset!")
else:
    print(f"\n⚠️ Total missing values: {missing_values.sum()}")

### 2.4 Visualisasi Distribusi Target (Cek Imbalance)

**⚠️ PENTING:** Langkah ini menentukan apakah kita perlu menggunakan parameter `scale_pos_weight` saat training.

Menurut modul:
- Jika proporsi kelas **seimbang** → Training normal
- Jika proporsi kelas **timpang (imbalanced)** → Gunakan `scale_pos_weight`

In [ ]:
# Hitung distribusi target
target_counts = df['target'].value_counts()
target_percentage = df['target'].value_counts(normalize=True) * 100

print("=== DISTRIBUSI TARGET ===")
print(f"\nKelas 0 (Tidak Sakit): {target_counts[0]} sampel ({target_percentage[0]:.2f}%)")
print(f"Kelas 1 (Sakit Jantung): {target_counts[1]} sampel ({target_percentage[1]:.2f}%)")

# Hitung rasio imbalance
imbalance_ratio = target_counts[0] / target_counts[1]
print(f"\nRasio Imbalance: {imbalance_ratio:.2f}")

# Tentukan status imbalance
if imbalance_ratio > 1.5 or imbalance_ratio < 0.67:
    print("\n⚠️ Dataset IMBALANCED! Akan menggunakan scale_pos_weight saat training.")
    is_imbalanced = True
else:
    print("\n✅ Dataset BALANCED! Training tanpa scale_pos_weight.")
    is_imbalanced = False

# Visualisasi dengan Bar Plot
plt.figure(figsize=(10, 5))

# Subplot 1: Count Plot
plt.subplot(1, 2, 1)
sns.countplot(data=df, x='target', palette='Set2')
plt.title('Distribusi Target Class', fontsize=14, fontweight='bold')
plt.xlabel('Target (0: Tidak Sakit, 1: Sakit Jantung)', fontsize=11)
plt.ylabel('Jumlah Sampel', fontsize=11)

# Tambahkan label angka di atas bar
for i, count in enumerate(target_counts):
    plt.text(i, count + 10, str(count), ha='center', fontweight='bold')

# Subplot 2: Pie Chart
plt.subplot(1, 2, 2)
plt.pie(target_counts, labels=['Tidak Sakit (0)', 'Sakit Jantung (1)'], 
        autopct='%1.1f%%', startangle=90, colors=['#90EE90', '#FFB6C1'])
plt.title('Proporsi Target Class', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

### 2.5 Korelasi Antar Fitur

Visualisasi korelasi untuk memahami hubungan antar fitur dan target variable.

In [ ]:
# Hitung korelasi
correlation_matrix = df.corr()

# Visualisasi dengan Heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            linewidths=0.5, cbar_kws={'shrink': 0.8})
plt.title('Correlation Matrix - Heart Disease Dataset', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Korelasi dengan target
print("\n=== KORELASI DENGAN TARGET (Sorted) ===")
target_correlation = correlation_matrix['target'].sort_values(ascending=False)
print(target_correlation)

---

## 3. Preprocessing Data

### 3.1 Strategi Preprocessing

Berdasarkan analisis EDA:
1. **Missing Values:** Tidak ada missing values → Tidak perlu imputation
2. **Encoding:** Dataset sudah dalam format numerik → Tidak perlu encoding tambahan
3. **Split Data:** Bagi menjadi 80% Training dan 20% Testing

In [ ]:
# Pisahkan fitur (X) dan target (y)
X = df.drop('target', axis=1)
y = df['target']

print("=== DIMENSI DATA ===")
print(f"Fitur (X): {X.shape}")
print(f"Target (y): {y.shape}")

print("\n=== NAMA FITUR ===")
print(list(X.columns))

### 3.2 Split Data (Training 80% & Testing 20%)

Menggunakan `train_test_split` dengan stratified sampling untuk mempertahankan proporsi kelas.

In [ ]:
# Split data dengan stratify untuk mempertahankan proporsi kelas
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,      # 20% untuk testing
    random_state=42,    # Reproducibility
    stratify=y          # Pertahankan proporsi kelas
)

print("=== HASIL SPLIT DATA ===")
print(f"Training Set: {X_train.shape[0]} sampel ({X_train.shape[0]/len(df)*100:.1f}%)")
print(f"Testing Set:  {X_test.shape[0]} sampel ({X_test.shape[0]/len(df)*100:.1f}%)")

print("\n=== DISTRIBUSI TARGET DI TRAINING SET ===")
print(y_train.value_counts())
print(f"\nProporsi: \n{y_train.value_counts(normalize=True) * 100}")

print("\n=== DISTRIBUSI TARGET DI TESTING SET ===")
print(y_test.value_counts())
print(f"\nProporsi: \n{y_test.value_counts(normalize=True) * 100}")

---

## 4. Implementasi XGBoost & Tuning

### 4.1 Baseline Model (XGBoost dengan Parameter Default)

Training model XGBoost dengan parameter default, dengan menambahkan `scale_pos_weight` jika dataset imbalanced.

In [ ]:
# Hitung scale_pos_weight jika dataset imbalanced
if is_imbalanced:
    negative_count = (y_train == 0).sum()
    positive_count = (y_train == 1).sum()
    scale_pos_weight = negative_count / positive_count
    print(f"⚠️ Dataset Imbalanced Detected!")
    print(f"Negative samples: {negative_count}")
    print(f"Positive samples: {positive_count}")
    print(f"scale_pos_weight = {scale_pos_weight:.2f}\n")
else:
    scale_pos_weight = 1
    print(f"✅ Dataset Balanced. scale_pos_weight = 1\n")

# Inisialisasi model XGBoost Baseline
xgb_baseline = XGBClassifier(
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss'
)

# Training model
print("🔄 Training XGBoost Baseline Model...")
xgb_baseline.fit(X_train, y_train)
print("✅ Training selesai!")

# Prediksi
y_pred_baseline = xgb_baseline.predict(X_test)
y_pred_proba_baseline = xgb_baseline.predict_proba(X_test)[:, 1]

# Evaluasi Baseline
accuracy_baseline = accuracy_score(y_test, y_pred_baseline)
roc_auc_baseline = roc_auc_score(y_test, y_pred_proba_baseline)

print(f"\n=== PERFORMA BASELINE MODEL ===")
print(f"Accuracy: {accuracy_baseline:.4f}")
print(f"ROC-AUC Score: {roc_auc_baseline:.4f}")

### 4.2 Hyperparameter Tuning

Eksperimen dengan beberapa kombinasi hyperparameter:
- `max_depth`: Mengontrol kedalaman pohon (kompleksitas model)
- `learning_rate`: Mengontrol learning rate (kecepatan konvergensi)

In [ ]:
# Definisikan kombinasi hyperparameter untuk tuning
tuning_params = [
    {'max_depth': 3, 'learning_rate': 0.1, 'name': 'Model 1 (Shallow & Fast)'},
    {'max_depth': 5, 'learning_rate': 0.1, 'name': 'Model 2 (Medium & Fast)'},
    {'max_depth': 7, 'learning_rate': 0.1, 'name': 'Model 3 (Deep & Fast)'},
    {'max_depth': 5, 'learning_rate': 0.05, 'name': 'Model 4 (Medium & Slow)'},
    {'max_depth': 5, 'learning_rate': 0.2, 'name': 'Model 5 (Medium & Very Fast)'},
]

# Dictionary untuk menyimpan hasil
tuning_results = []

print("🔄 Mulai Hyperparameter Tuning...\n")

for params in tuning_params:
    # Training model
    model = XGBClassifier(
        max_depth=params['max_depth'],
        learning_rate=params['learning_rate'],
        scale_pos_weight=scale_pos_weight,
        random_state=42,
        eval_metric='logloss'
    )
    
    model.fit(X_train, y_train)
    
    # Prediksi
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Evaluasi
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    
    # Simpan hasil
    tuning_results.append({
        'Model': params['name'],
        'max_depth': params['max_depth'],
        'learning_rate': params['learning_rate'],
        'Accuracy': accuracy,
        'ROC-AUC': roc_auc
    })
    
    print(f"✅ {params['name']}")
    print(f"   max_depth={params['max_depth']}, learning_rate={params['learning_rate']}")
    print(f"   Accuracy: {accuracy:.4f}, ROC-AUC: {roc_auc:.4f}\n")

# Konversi ke DataFrame untuk visualisasi
results_df = pd.DataFrame(tuning_results)

print("\n=== SUMMARY HYPERPARAMETER TUNING ===")
print(results_df.to_string(index=False))

### 4.3 Visualisasi Hasil Tuning

In [ ]:
# Visualisasi perbandingan hasil tuning
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Accuracy
axes[0].barh(results_df['Model'], results_df['Accuracy'], color='skyblue')
axes[0].set_xlabel('Accuracy', fontsize=12)
axes[0].set_title('Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_xlim([0.7, 1.0])

# Tambahkan nilai di bar
for i, v in enumerate(results_df['Accuracy']):
    axes[0].text(v + 0.01, i, f'{v:.4f}', va='center')

# Plot 2: ROC-AUC
axes[1].barh(results_df['Model'], results_df['ROC-AUC'], color='salmon')
axes[1].set_xlabel('ROC-AUC Score', fontsize=12)
axes[1].set_title('ROC-AUC Comparison', fontsize=14, fontweight='bold')
axes[1].set_xlim([0.7, 1.0])

# Tambahkan nilai di bar
for i, v in enumerate(results_df['ROC-AUC']):
    axes[1].text(v + 0.01, i, f'{v:.4f}', va='center')

plt.tight_layout()
plt.show()

# Pilih model terbaik berdasarkan ROC-AUC
best_model_idx = results_df['ROC-AUC'].idxmax()
best_model_info = results_df.iloc[best_model_idx]

print(f"\n🏆 MODEL TERBAIK: {best_model_info['Model']}")
print(f"   max_depth = {best_model_info['max_depth']}")
print(f"   learning_rate = {best_model_info['learning_rate']}")
print(f"   Accuracy = {best_model_info['Accuracy']:.4f}")
print(f"   ROC-AUC = {best_model_info['ROC-AUC']:.4f}")

### 4.4 Training Final Model dengan Best Hyperparameters

In [ ]:
# Training final model dengan hyperparameter terbaik
xgb_final = XGBClassifier(
    max_depth=int(best_model_info['max_depth']),
    learning_rate=best_model_info['learning_rate'],
    scale_pos_weight=scale_pos_weight,
    random_state=42,
    eval_metric='logloss',
    n_estimators=100  # Jumlah pohon
)

print("🔄 Training Final XGBoost Model dengan Best Hyperparameters...")
xgb_final.fit(X_train, y_train)
print("✅ Training selesai!")

# Prediksi final
y_pred_final = xgb_final.predict(X_test)
y_pred_proba_final = xgb_final.predict_proba(X_test)[:, 1]

### 4.5 Threshold Optimization (Imbalance Handling)

**⚠️ PENTING:** Untuk dataset imbalanced, default threshold 0.5 tidak selalu optimal.
Kita perlu mencari threshold yang menghasilkan F1-Score terbaik.


In [ ]:
# ===== THRESHOLD TUNING =====
# Untuk dataset imbalanced, kita perlu mencari threshold optimal

print("="*70)
print("THRESHOLD TUNING untuk Imbalanced Dataset")
print("="*70)

# Import additional metrics
from sklearn.metrics import precision_score, recall_score, f1_score

# Coba berbagai threshold values
thresholds_to_test = np.arange(0.1, 1.0, 0.1)
threshold_results = []

print("\nTesting berbagai threshold values...\n")

for thresh in thresholds_to_test:
    # Apply threshold
    y_pred_thresh = (y_pred_proba_final >= thresh).astype(int)
    
    # Hitung metrics
    acc = accuracy_score(y_test, y_pred_thresh)
    prec = precision_score(y_test, y_pred_thresh, zero_division=0)
    rec = recall_score(y_test, y_pred_thresh, zero_division=0)
    f1 = f1_score(y_test, y_pred_thresh, zero_division=0)
    roc_auc_thresh = roc_auc_score(y_test, y_pred_proba_final)
    
    threshold_results.append({
        'Threshold': thresh,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1-Score': f1,
        'ROC-AUC': roc_auc_thresh
    })
    
    print(f"Threshold {thresh:.1f}: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f}, F1={f1:.4f}")

# Konversi ke DataFrame
threshold_df = pd.DataFrame(threshold_results)

# Pilih threshold dengan F1-Score terbaik
best_thresh_idx = threshold_df['F1-Score'].idxmax()
optimal_threshold = threshold_df.iloc[best_thresh_idx]['Threshold']
best_f1 = threshold_df.iloc[best_thresh_idx]['F1-Score']

print("\n" + "="*70)
print(f"🎯 OPTIMAL THRESHOLD: {optimal_threshold:.1f}")
print(f"   F1-Score pada threshold ini: {best_f1:.4f}")
print(f"   Accuracy: {threshold_df.iloc[best_thresh_idx]['Accuracy']:.4f}")
print(f"   Recall: {threshold_df.iloc[best_thresh_idx]['Recall']:.4f}")
print(f"   Precision: {threshold_df.iloc[best_thresh_idx]['Precision']:.4f}")
print("="*70)

# Visualisasi perbandingan threshold
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Plot 1: Accuracy
axes[0, 0].plot(threshold_df['Threshold'], threshold_df['Accuracy'], 'o-', linewidth=2, markersize=8)
axes[0, 0].axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal ({optimal_threshold:.1f})')
axes[0, 0].set_xlabel('Threshold')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].set_title('Accuracy vs Threshold')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Plot 2: Precision & Recall
axes[0, 1].plot(threshold_df['Threshold'], threshold_df['Precision'], 'o-', label='Precision', linewidth=2, markersize=8)
axes[0, 1].plot(threshold_df['Threshold'], threshold_df['Recall'], 's-', label='Recall', linewidth=2, markersize=8)
axes[0, 1].axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal ({optimal_threshold:.1f})')
axes[0, 1].set_xlabel('Threshold')
axes[0, 1].set_ylabel('Score')
axes[0, 1].set_title('Precision & Recall vs Threshold')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Plot 3: F1-Score
axes[1, 0].plot(threshold_df['Threshold'], threshold_df['F1-Score'], 'o-', color='green', linewidth=2, markersize=8)
axes[1, 0].axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal ({optimal_threshold:.1f})')
axes[1, 0].scatter([optimal_threshold], [best_f1], color='red', s=200, zorder=5, label='Peak')
axes[1, 0].set_xlabel('Threshold')
axes[1, 0].set_ylabel('F1-Score')
axes[1, 0].set_title('F1-Score vs Threshold (MOST IMPORTANT for Imbalanced)')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Plot 4: All metrics
axes[1, 1].plot(threshold_df['Threshold'], threshold_df['Accuracy'], 'o-', label='Accuracy', linewidth=2)
axes[1, 1].plot(threshold_df['Threshold'], threshold_df['Precision'], 's-', label='Precision', linewidth=2)
axes[1, 1].plot(threshold_df['Threshold'], threshold_df['Recall'], '^-', label='Recall', linewidth=2)
axes[1, 1].plot(threshold_df['Threshold'], threshold_df['F1-Score'], 'D-', label='F1-Score', linewidth=2)
axes[1, 1].axvline(x=optimal_threshold, color='red', linestyle='--', label=f'Optimal ({optimal_threshold:.1f})')
axes[1, 1].set_xlabel('Threshold')
axes[1, 1].set_ylabel('Score')
axes[1, 1].set_title('All Metrics vs Threshold')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()


### 4.5 Perbandingan dengan Decision Tree (Bonus)

Membuktikan keunggulan XGBoost dibandingkan Decision Tree biasa.

### 4.6 Final Predictions dengan Optimal Threshold

Sekarang kita menggunakan optimal threshold untuk prediksi final, bukan default 0.5.


In [ ]:
# Gunakan optimal threshold untuk prediksi final
print("="*70)
print("FINAL PREDICTIONS DENGAN OPTIMAL THRESHOLD")
print("="*70)

y_pred_final_optimized = (y_pred_proba_final >= optimal_threshold).astype(int)

print(f"\nMenggunakan threshold: {optimal_threshold:.1f}")
print(f"(Bukan default 0.5)\n")

# Hitung metrics dengan optimal threshold
from sklearn.metrics import precision_score, recall_score, f1_score

acc_opt = accuracy_score(y_test, y_pred_final_optimized)
prec_opt = precision_score(y_test, y_pred_final_optimized)
rec_opt = recall_score(y_test, y_pred_final_optimized)
f1_opt = f1_score(y_test, y_pred_final_optimized)

print("PERFORMA DENGAN OPTIMAL THRESHOLD:")
print(f"  Accuracy:  {acc_opt:.4f}")
print(f"  Precision: {prec_opt:.4f}")
print(f"  Recall:    {rec_opt:.4f}")
print(f"  F1-Score:  {f1_opt:.4f}\n")

print("PERFORMA DENGAN DEFAULT THRESHOLD (0.5):")
print(f"  Accuracy:  {accuracy_score(y_test, y_pred_final):.4f}")
print(f"  Precision: {precision_score(y_test, y_pred_final):.4f}")
print(f"  Recall:    {recall_score(y_test, y_pred_final):.4f}")
print(f"  F1-Score:  {f1_score(y_test, y_pred_final):.4f}\n")

print("="*70)
print(f"✅ Optimal threshold memberikan F1-Score yang lebih baik!")
print(f"   untuk menangani imbalance dalam dataset")
print("="*70)


In [ ]:
# Training Decision Tree untuk perbandingan
dt_model = DecisionTreeClassifier(random_state=42)

print("🔄 Training Decision Tree Model...")
dt_model.fit(X_train, y_train)
print("✅ Training selesai!")

# Prediksi Decision Tree
y_pred_dt = dt_model.predict(X_test)
y_pred_proba_dt = dt_model.predict_proba(X_test)[:, 1]

# Evaluasi Decision Tree
accuracy_dt = accuracy_score(y_test, y_pred_dt)
roc_auc_dt = roc_auc_score(y_test, y_pred_proba_dt)

# Evaluasi Final XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_final)
roc_auc_xgb = roc_auc_score(y_test, y_pred_proba_final)

# Perbandingan
print("\n" + "="*60)
print("PERBANDINGAN: XGBoost vs Decision Tree")
print("="*60)
print(f"\n{'Metric':<20} {'XGBoost':<15} {'Decision Tree':<15} {'Improvement'}")
print("-" * 60)
print(f"{'Accuracy':<20} {accuracy_xgb:<15.4f} {accuracy_dt:<15.4f} {(accuracy_xgb - accuracy_dt)*100:+.2f}%")
print(f"{'ROC-AUC Score':<20} {roc_auc_xgb:<15.4f} {roc_auc_dt:<15.4f} {(roc_auc_xgb - roc_auc_dt)*100:+.2f}%")
print("="*60)

if accuracy_xgb > accuracy_dt:
    print("\n✅ XGBoost mengungguli Decision Tree!")
else:
    print("\n⚠️ Decision Tree lebih baik pada dataset ini.")

---

## 5. Evaluasi Model

### 5.1 Confusion Matrix

Visualisasi confusion matrix untuk melihat distribusi prediksi benar dan salah.

In [ ]:
# Hitung Confusion Matrix dengan optimal threshold
cm = confusion_matrix(y_test, y_pred_final_optimized)

# Visualisasi Confusion Matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=['Predicted: No Disease (0)', 'Predicted: Disease (1)'],
            yticklabels=['Actual: No Disease (0)', 'Actual: Disease (1)'])
plt.title(f'Confusion Matrix - XGBoost Model (Threshold: {optimal_threshold:.1f})', fontsize=16, fontweight='bold', pad=20)
plt.ylabel('Actual', fontsize=12)
plt.xlabel('Predicted', fontsize=12)
plt.tight_layout()
plt.show()

# Interpretasi
tn, fp, fn, tp = cm.ravel()
print("\n=== INTERPRETASI CONFUSION MATRIX ===")
print(f"True Negative (TN):  {tn} → Benar prediksi TIDAK SAKIT")
print(f"False Positive (FP): {fp} → Salah prediksi SAKIT (padahal tidak)")
print(f"False Negative (FN): {fn} → Salah prediksi TIDAK SAKIT (padahal sakit) ⚠️")
print(f"True Positive (TP):  {tp} → Benar prediksi SAKIT")
print(f"\n📌 Catatan: Dengan optimal threshold {optimal_threshold:.1f}, model lebih")
print(f"   sensitif mendeteksi kasus positif (lebih sedikit False Negative)")


### 5.2 Classification Report

Menampilkan Precision, Recall, F1-Score untuk setiap kelas.

In [ ]:
# Classification Report
print("=== CLASSIFICATION REPORT (dengan optimal threshold) ===")
print(classification_report(y_test, y_pred_final_optimized, 
                          target_names=['No Disease (0)', 'Disease (1)']))

print("\n📊 Penjelasan Metrik:")
print("- Precision: Dari semua prediksi positif, berapa yang benar?")
print("- Recall: Dari semua kasus positif, berapa yang berhasil diprediksi?")
print("- F1-Score: Harmonic mean dari Precision dan Recall")
print("- Support: Jumlah sampel aktual di setiap kelas")
print("\n⚠️ Untuk medical diagnosis, RECALL lebih penting dari Precision")
print("   karena kita ingin mendeteksi sebanyak mungkin kasus positif (tidak boleh terlewat)")


### 5.3 ROC-AUC Curve

Visualisasi kurva ROC untuk mengevaluasi kemampuan model membedakan kelas.

In [ ]:
# Hitung ROC Curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_final)
roc_auc = roc_auc_score(y_test, y_pred_proba_final)

# Plot ROC Curve
plt.figure(figsize=(10, 7))
plt.plot(fpr, tpr, color='darkorange', lw=2, 
         label=f'XGBoost (AUC = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', 
         label='Random Classifier (AUC = 0.5000)')

# Tambahkan marker untuk optimal threshold
# Cari FPR dan TPR yang sesuai dengan optimal threshold
idx_optimal = np.argmin(np.abs(thresholds - optimal_threshold))
optimal_fpr = fpr[idx_optimal]
optimal_tpr = tpr[idx_optimal]

plt.scatter(optimal_fpr, optimal_tpr, marker='o', color='red', s=200, 
           label=f'Optimal Threshold ({optimal_threshold:.1f})', zorder=5)

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=13)
plt.ylabel('True Positive Rate', fontsize=13)
plt.title('ROC Curve - XGBoost Model (with Optimal Threshold)', fontsize=16, fontweight='bold')
plt.legend(loc='lower right', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\n📈 ROC-AUC Score: {roc_auc:.4f}")
print(f"\nOptimal Threshold: {optimal_threshold:.1f}")
print(f"  - True Positive Rate (Recall): {optimal_tpr:.4f}")
print(f"  - False Positive Rate: {optimal_fpr:.4f}")

print("\nInterpretasi:")
if roc_auc >= 0.90:
    print("✅ Excellent: Model sangat baik dalam membedakan kelas!")
elif roc_auc >= 0.80:
    print("✅ Good: Model baik dalam membedakan kelas.")
elif roc_auc >= 0.70:
    print("⚠️ Fair: Model cukup baik, masih bisa ditingkatkan.")
else:
    print("❌ Poor: Model perlu perbaikan signifikan.")


---

## 6. Feature Importance & Model Interpretability

### 6.1 Visualisasi Feature Importance

**⚠️ WAJIB:** Analisis fitur mana yang paling dominan mempengaruhi prediksi model.

In [ ]:
# Plot Feature Importance dengan XGBoost built-in function
plt.figure(figsize=(10, 8))
plot_importance(xgb_final, max_num_features=13, importance_type='weight', 
                title='Feature Importance (Weight) - XGBoost Model')
plt.tight_layout()
plt.show()

### 6.2 Feature Importance dengan Bar Chart (Custom)

Membuat visualisasi yang lebih detail dan terurut.

In [ ]:
# Ekstrak feature importance
feature_importance = xgb_final.feature_importances_
features = X.columns

# Buat DataFrame untuk sorting
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importance
}).sort_values('Importance', ascending=False)

print("=== TOP 10 MOST IMPORTANT FEATURES ===")
print(importance_df.head(10).to_string(index=False))

# Visualisasi dengan Bar Chart
plt.figure(figsize=(12, 8))
colors = plt.cm.viridis(np.linspace(0, 1, len(importance_df)))
plt.barh(importance_df['Feature'], importance_df['Importance'], color=colors)
plt.xlabel('Importance Score', fontsize=13)
plt.ylabel('Features', fontsize=13)
plt.title('Feature Importance - Ranked by XGBoost', fontsize=16, fontweight='bold')
plt.gca().invert_yaxis()  # Feature terpenting di atas
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### 6.3 Analisis Feature Importance (Interpretability)

**⚠️ WAJIB:** Jelaskan apakah fitur yang dianggap penting oleh model masuk akal secara logika medis/bisnis.

In [ ]:
# Deskripsi fitur untuk interpretasi
feature_description = {
    'age': 'Usia pasien (tahun)',
    'sex': 'Jenis kelamin (0=female, 1=male)',
    'cp': 'Tipe nyeri dada (0-3)',
    'trestbps': 'Tekanan darah saat istirahat (mmHg)',
    'chol': 'Kolesterol serum (mg/dl)',
    'fbs': 'Gula darah puasa > 120 mg/dl',
    'restecg': 'Hasil EKG saat istirahat',
    'thalach': 'Detak jantung maksimum',
    'exang': 'Angina akibat olahraga',
    'oldpeak': 'Depresi ST akibat olahraga',
    'slope': 'Kemiringan segmen ST',
    'ca': 'Jumlah pembuluh darah (0-3)',
    'thal': 'Thalassemia (1=normal, 2=fixed defect, 3=reversable defect)'
}

print("=" * 80)
print("ANALISIS INTERPRETABILITY: Apakah Feature Importance Masuk Akal?")
print("=" * 80)

top_5_features = importance_df.head(5)

print("\n🔍 TOP 5 FITUR PALING PENTING:\n")
for idx, (i, row) in enumerate(top_5_features.iterrows(), 1):
    feature_name = row['Feature']
    importance_score = row['Importance']
    description = feature_description.get(feature_name, 'Deskripsi tidak tersedia')
    
    print(f"{idx}. {feature_name.upper()} (Score: {importance_score:.4f})")
    print(f"   Deskripsi: {description}")
    print(f"   Korelasi dengan target: {correlation_matrix.loc[feature_name, 'target']:.4f}")
    print()

print("\n" + "="*80)
print("💡 KESIMPULAN INTERPRETABILITY:")
print("="*80)
print("""
Berdasarkan analisis Feature Importance di atas, model XGBoost menunjukkan bahwa:

1. Fitur-fitur yang dianggap penting oleh model MASUK AKAL secara medis karena:
   - Fitur seperti 'thalach' (detak jantung maksimum), 'cp' (tipe nyeri dada), 
     dan 'oldpeak' (depresi ST) adalah indikator klinis yang umum digunakan 
     dokter untuk mendiagnosis penyakit jantung.
   
2. Korelasi dengan target variable mendukung interpretasi ini:
   - Fitur dengan importance tinggi cenderung memiliki korelasi kuat (positif/negatif)
     dengan target variable.

3. Model tidak hanya "belajar pola acak", tetapi menangkap hubungan medis yang valid:
   - Hal ini membuktikan bahwa XGBoost tidak overfitting dan dapat diandalkan
     untuk aplikasi klinis (dengan supervisi ahli medis).

⚠️ CATATAN PENTING:
- Model ini adalah alat bantu prediksi, BUKAN pengganti diagnosis medis profesional.
- Keputusan klinis harus selalu melibatkan tenaga medis yang berkompeten.
""")
print("="*80)

---

## 7. Kesimpulan & Rekomendasi

### 7.1 Summary Hasil Proyek

In [ ]:
print("=" * 80)
print("RINGKASAN HASIL PROYEK")
print("=" * 80)

print("\n📊 DATASET:")
print(f"   - Total Sampel: {len(df)}")
print(f"   - Jumlah Fitur: {X.shape[1]}")
print(f"   - Training Set: {X_train.shape[0]} sampel")
print(f"   - Testing Set: {X_test.shape[0]} sampel")
print(f"   - Status Imbalance: {'IMBALANCED' if is_imbalanced else 'BALANCED'}")

print("\n🎯 PERFORMA MODEL TERBAIK (XGBoost):")
print(f"   - Accuracy: {accuracy_xgb:.4f}")
print(f"   - ROC-AUC Score: {roc_auc_xgb:.4f}")
print(f"   - Hyperparameters:")
print(f"     • max_depth = {int(best_model_info['max_depth'])}")
print(f"     • learning_rate = {best_model_info['learning_rate']}")
print(f"     • scale_pos_weight = {scale_pos_weight:.2f}")

print("\n📈 PERBANDINGAN MODEL:")
print(f"   XGBoost vs Decision Tree:")
print(f"   - Improvement Accuracy: {(accuracy_xgb - accuracy_dt)*100:+.2f}%")
print(f"   - Improvement ROC-AUC: {(roc_auc_xgb - roc_auc_dt)*100:+.2f}%")

print("\n🔍 TOP 3 FITUR PALING PENTING:")
for idx, (i, row) in enumerate(importance_df.head(3).iterrows(), 1):
    print(f"   {idx}. {row['Feature']} (Score: {row['Importance']:.4f})")

print("\n" + "="*80)
print("✅ PROYEK BERHASIL DISELESAIKAN!")
print("="*80)

### 7.2 Rekomendasi untuk Pengembangan Lebih Lanjut

1. **Feature Engineering:**
   - Buat fitur interaksi (misal: age × cholesterol)
   - Binning untuk fitur numerik kontinu

2. **Advanced Tuning:**
   - Gunakan GridSearchCV atau RandomizedSearchCV
   - Tuning parameter tambahan: n_estimators, subsample, colsample_bytree

3. **Ensemble Methods:**
   - Kombinasi XGBoost dengan model lain (Voting Classifier)
   - Stacking dengan meta-learner

4. **Deployment:**
   - Simpan model terbaik dengan pickle/joblib
   - Buat API untuk prediksi real-time

5. **Interpretability:**
   - Gunakan SHAP values untuk explainability yang lebih detail
   - Analisis partial dependence plots

---

## 📝 Catatan untuk Pengumpulan

**Checklist Deliverables:**
- ✅ Load Data & EDA (Cek Imbalance)
- ✅ Preprocessing
- ✅ XGBoost Training (termasuk scale_pos_weight)
- ✅ Hyperparameter Tuning
- ✅ Evaluasi (Confusion Matrix & ROC Curve)
- ✅ Visualisasi Feature Importance
- ✅ Analisis Interpretability

**Format Pengumpulan:**
- File: `TA14_XGBoost_Heart_Disease.ipynb`
- Platform: GitHub Repository atau Folder ZIP

---

*Proyek ini dibuat untuk memenuhi Tugas Akhir 14 - Praktikum Machine Learning*  
*Semester 5 - Tahun Akademik 2025/2026*

### 8.5 Template Deployment API (Flask)

**Contoh kode untuk deployment menggunakan Flask API:**

```python
# save as: app.py

from flask import Flask, request, jsonify
import joblib
import numpy as np

app = Flask(__name__)

# Load model saat aplikasi start
model = joblib.load('xgboost_heart_disease_model.joblib')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Ambil data dari request
        data = request.get_json()
        
        # Convert ke array dengan urutan fitur yang benar
        features = np.array([[
            data['age'], data['sex'], data['cp'], data['trestbps'],
            data['chol'], data['fbs'], data['restecg'], data['thalach'],
            data['exang'], data['oldpeak'], data['slope'], data['ca'],
            data['thal']
        ]])
        
        # Prediksi
        prediction = model.predict(features)[0]
        prediction_proba = model.predict_proba(features)[0]
        
        # Return hasil
        return jsonify({
            'prediction': int(prediction),
            'interpretation': 'Heart Disease Risk' if prediction == 1 else 'No Risk',
            'probability': {
                'no_disease': float(prediction_proba[0]),
                'disease': float(prediction_proba[1])
            }
        })
    
    except Exception as e:
        return jsonify({'error': str(e)}), 400

@app.route('/health', methods=['GET'])
def health():
    return jsonify({'status': 'OK', 'model': 'XGBoost Heart Disease Classifier'})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
```

**Cara menjalankan:**
```bash
pip install flask
python app.py
```

**Cara testing API:**
```bash
curl -X POST http://localhost:5000/predict \
  -H "Content-Type: application/json" \
  -d '{"age":55,"sex":1,"cp":2,"trestbps":140,"chol":250,"fbs":0,"restecg":1,"thalach":150,"exang":1,"oldpeak":2.5,"slope":1,"ca":1,"thal":3}'
```

In [ ]:
# Contoh data pasien baru
new_patient_data = {
    'age': 55,
    'sex': 1,           # Male
    'cp': 2,            # Chest pain type
    'trestbps': 140,    # Blood pressure
    'chol': 250,        # Cholesterol
    'fbs': 0,           # Fasting blood sugar
    'restecg': 1,       # ECG results
    'thalach': 150,     # Max heart rate
    'exang': 1,         # Exercise induced angina
    'oldpeak': 2.5,     # ST depression
    'slope': 1,         # Slope of ST segment
    'ca': 1,            # Number of major vessels
    'thal': 3           # Thalassemia
}

# Konversi ke DataFrame
new_patient_df = pd.DataFrame([new_patient_data])

print("=== DATA PASIEN BARU ===")
print(new_patient_df.T)

# Prediksi dengan model yang di-load dan optimal threshold
prediction_proba = loaded_model_joblib.predict_proba(new_patient_df)[0]
prediction = 1 if prediction_proba[1] >= loaded_optimal_threshold else 0

print("\n" + "="*60)
print("HASIL PREDIKSI (dengan optimal threshold):")
print("="*60)
print(f"\nPrediksi: {prediction}")
print(f"Interpretasi: {'⚠️ BERISIKO PENYAKIT JANTUNG' if prediction == 1 else '✅ TIDAK BERISIKO'}")
print(f"\nProbabilitas:")
print(f"  - Tidak Sakit (0): {prediction_proba[0]:.2%}")
print(f"  - Sakit Jantung (1): {prediction_proba[1]:.2%}")
print(f"\nThreshold: {loaded_optimal_threshold:.1f}")
print("="*60)
print("\n⚠️ DISCLAIMER: Hasil prediksi ini hanya untuk referensi.")
print("   Konsultasikan dengan tenaga medis profesional untuk diagnosis resmi.")


### 8.4 Contoh Prediksi untuk Data Baru

Simulasi cara menggunakan model untuk memprediksi data pasien baru.

In [ ]:
# Prediksi dengan model original (menggunakan optimal threshold)
y_pred_proba_test = xgb_final.predict_proba(X_test[:5])[:, 1]
y_pred_original = (y_pred_proba_test >= optimal_threshold).astype(int)

# Prediksi dengan model yang di-load (menggunakan optimal threshold dari metadata)
y_pred_proba_loaded = loaded_model_joblib.predict_proba(X_test[:5])[:, 1]
y_pred_loaded = (y_pred_proba_loaded >= loaded_optimal_threshold).astype(int)

# Verifikasi
print("=== VERIFIKASI PREDIKSI ===\n")
print(f"Original threshold: {optimal_threshold:.1f}")
print(f"Loaded threshold:   {loaded_optimal_threshold:.1f}\n")
print("Prediksi dari model original:", y_pred_original)
print("Prediksi dari model loaded:  ", y_pred_loaded)
print("\nApakah prediksi sama?", np.array_equal(y_pred_original, y_pred_loaded))

if np.array_equal(y_pred_original, y_pred_loaded):
    print("\n✅ VERIFIKASI BERHASIL! Model dapat digunakan untuk deployment.")
    print(f"   Optimal threshold: {loaded_optimal_threshold:.1f}")
else:
    print("\n❌ VERIFIKASI GAGAL! Ada perbedaan prediksi.")


### 8.3 Verifikasi Model dengan Prediksi

Memastikan model yang di-load memberikan hasil prediksi yang sama dengan model original.

In [ ]:
# Load model dengan pickle
print("🔄 Loading model dengan pickle...")
with open(model_filename_pkl, 'rb') as file:
    loaded_model_pkl = pickle.load(file)
print("✅ Model berhasil di-load dengan pickle!")

# Load model dengan joblib (RECOMMENDED untuk production)
print("\n🔄 Loading model dengan joblib...")
loaded_model_joblib = joblib.load(model_filename_joblib)
print("✅ Model berhasil di-load dengan joblib!")

# Load metadata
print("\n🔄 Loading metadata...")
with open(metadata_filename, 'rb') as file:
    loaded_metadata = pickle.load(file)
print("✅ Metadata berhasil di-load!")

# Extract optimal threshold dari metadata
loaded_optimal_threshold = loaded_metadata['hyperparameters']['optimal_threshold']

# Tampilkan metadata
print("\n" + "="*60)
print("INFORMASI MODEL YANG DI-LOAD:")
print("="*60)
print(f"Model Name: {loaded_metadata['model_name']}")
print(f"Training Date: {loaded_metadata['training_date']}")
print(f"Dataset Size: {loaded_metadata['dataset_size']}")
print(f"Number of Features: {len(loaded_metadata['features'])}")
print(f"Accuracy: {loaded_metadata['accuracy']:.4f}")
print(f"Precision: {loaded_metadata['precision']:.4f}")
print(f"Recall: {loaded_metadata['recall']:.4f}")
print(f"F1-Score: {loaded_metadata['f1_score']:.4f}")
print(f"ROC-AUC: {loaded_metadata['roc_auc']:.4f}")
print(f"\nHyperparameters:")
for key, value in loaded_metadata['hyperparameters'].items():
    print(f"  - {key}: {value}")
print(f"\nClass Distribution:")
for key, value in loaded_metadata['class_distribution'].items():
    print(f"  - {key}: {value}")
print("="*60)


### 8.2 Load Model Kembali (Testing)

Menguji apakah model yang disimpan dapat di-load kembali dengan benar.

In [ ]:
import pickle
import joblib
from datetime import datetime

# Buat folder untuk menyimpan model (jika belum ada)
import os
model_dir = '/content/drive/MyDrive/ML_Models'
os.makedirs(model_dir, exist_ok=True)

# Simpan model menggunakan pickle
model_filename_pkl = f'{model_dir}/xgboost_heart_disease_model.pkl'
with open(model_filename_pkl, 'wb') as file:
    pickle.dump(xgb_final, file)
print(f"✅ Model disimpan dengan pickle: {model_filename_pkl}")

# Simpan model menggunakan joblib (lebih efisien untuk model besar)
model_filename_joblib = f'{model_dir}/xgboost_heart_disease_model.joblib'
joblib.dump(xgb_final, model_filename_joblib)
print(f"✅ Model disimpan dengan joblib: {model_filename_joblib}")

# Simpan metadata model - SEKARANG TERMASUK OPTIMAL THRESHOLD
model_metadata = {
    'model_name': 'XGBoost Heart Disease Classifier',
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_size': len(df),
    'features': list(X.columns),
    'accuracy': acc_opt,
    'precision': prec_opt,
    'recall': rec_opt,
    'f1_score': f1_opt,
    'roc_auc': roc_auc,
    'hyperparameters': {
        'max_depth': int(best_model_info['max_depth']),
        'learning_rate': best_model_info['learning_rate'],
        'scale_pos_weight': scale_pos_weight,
        'n_estimators': 100,
        'optimal_threshold': float(optimal_threshold)  # 🔥 PENTING: Simpan optimal threshold
    },
    'is_imbalanced': is_imbalanced,
    'class_distribution': {
        'class_0_healthy': int((y == 0).sum()),
        'class_1_disease': int((y == 1).sum()),
        'imbalance_ratio': float((y == 0).sum() / (y == 1).sum())
    }
}

metadata_filename = f'{model_dir}/model_metadata.pkl'
with open(metadata_filename, 'wb') as file:
    pickle.dump(model_metadata, file)
print(f"✅ Metadata disimpan: {metadata_filename}")

print("\n" + "="*60)
print("MODEL BERHASIL DISIMPAN!")
print("="*60)
print(f"\n📊 Model Performance:")
print(f"   Accuracy:  {acc_opt:.4f}")
print(f"   Precision: {prec_opt:.4f}")
print(f"   Recall:    {rec_opt:.4f}")
print(f"   F1-Score:  {f1_opt:.4f}")
print(f"   ROC-AUC:   {roc_auc:.4f}")
print(f"\n⚙️ Key Hyperparameters:")
print(f"   scale_pos_weight:  {scale_pos_weight:.2f}")
print(f"   optimal_threshold: {optimal_threshold:.1f}")
print(f"\n💾 Files created:")
print(f"   - {model_filename_joblib} (model)")
print(f"   - {metadata_filename} (metadata with optimal_threshold)")


---

## 8. Simpan Model untuk Deployment

### 8.1 Menyimpan Model

Model yang sudah di-training perlu disimpan agar dapat digunakan untuk deployment atau prediksi di kemudian hari tanpa perlu training ulang.